In [1]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import soundfile
from scipy import signal
import IPython.display as ipd
import librosa, librosa.display
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio
from functions import plot_spectogram,preprocess,plot_wave,calculate_spectogram,normalize_audio
from scipy.io import wavfile
from audio_globals import n_mels,slice_lenght,overlap
import multiprocessing



# Load data

In [2]:
audio_data_df = pd.read_csv("audio_data.csv")
class_counts = audio_data_df["label"].value_counts()
threshold_min = class_counts.mean()-100  
threshold_max = class_counts.mean()+50
# Filter out classes with counts less than the threshold
filtered_df = audio_data_df[audio_data_df["label"].isin(class_counts[class_counts <= threshold_max].index)]
filtered_df = filtered_df[filtered_df["label"].isin(class_counts[class_counts >= threshold_min].index)]
filtered_df.to_csv("filtered_audio_data.csv", index=False) 

In [3]:
filtered_df= pd.read_csv("filtered_audio_data.csv")


In [4]:
filtered_df['label'].dtype

dtype('int64')

In [5]:
import h5py

def save_to_hdf5(batch, filename):
    with h5py.File(filename, 'a') as h5file:
        for i, row in batch.iterrows():
            grp = h5file.create_group(str(i))
            grp.create_dataset("label", data=np.array(row['label'], dtype='int64'))
            grp.create_dataset("path", data=np.string_(row['path']))
            grp.create_dataset("spec_data", data=row['spec_data'])

def process_audio_data(row, max_slices=10):
    sampling_rate, data = wavfile.read(row["path"])
    spectogram_blocks = []
    for start in range(0, len(data), slice_lenght):
        end = min(start + slice_lenght, len(data))
        block = data[start:end]
        spectogram_block_single, _ = preprocess(block, row['label'], sampling_rate, n_mels, slice_lenght)
        spectogram_blocks.append(spectogram_block_single.numpy())
        if len(spectogram_blocks) >= max_slices:
            break
    return spectogram_blocks

def batch_generator(df, batch_size):
    for i in range(0, len(df), batch_size):
        batch_df = df[i:i+batch_size]
        batch_df['spec_data'] = batch_df.apply(lambda row: process_audio_data(row), axis=1)
        yield batch_df
def save_to_csv(batch, filename, mode='a'):
    # Append batch to a CSV file
    batch.to_csv(filename, mode=mode, header=not bool(mode == 'a'), index=False)

# Define batch size based on your memory constraints
batch_size = 100
output_filename = "processed_data.h5"
i = 0
for batch in batch_generator(filtered_df, batch_size):
    processed_batch = batch # Assuming batch is already processed
    save_to_hdf5(processed_batch, output_filename)
    print("Batch done:",i)
    i+=1
    
print("Data processing and saving completed.")

FileNotFoundError: [Errno 2] No such file or directory: '/tmp\\sound_datasets\\fsd50k\\FSD50K.dev_audio/329135.wav'

# Train test split

In [12]:
train, validate, test = np.split(filtered_df.sample(frac=1, random_state=42), [int(.6*len(filtered_df)), int(.8*len(filtered_df))])

In [50]:
def load_from_hdf5(filename):
    with h5py.File(filename, 'r') as h5file:
        all_data = []
        for group_key in h5file.keys():
            group = h5file[group_key]
            data = {
                # Load label as int64
                'label': np.array(group['label']).astype('int64'),
                'path': np.array(group['path']).tobytes().decode('utf-8'),
                'spec_data': np.array(group['spec_data'])
            }
            all_data.append(data)
        return pd.DataFrame(all_data)

loaded_data = load_from_hdf5("processed_data.h5")

KeyboardInterrupt: 

In [49]:
loaded_data.head(n=50)

first_row = loaded_data.iloc[1]


first_row['label']

'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'